In [34]:
import numpy as np
import time

start = time.time()

# 난수 생성 시드 고정
np.random.seed(42)

# 데이터 설정
S_i = [10450, 10450, 8360, 8360]  # 포화교통류율
v = [8880, 6595, 2964, 1309]      # 도착 교통량
C_min = 30                        # 최소 신호주기
C_max = 240                       # 최대 신호주기
num_phases = 3                    # 현시 수

# 목적 함수 정의
def objective_function(params):
    g_i = params[:num_phases]
    L_i = params[num_phases:]
    C = sum(g_i) + sum(L_i)
    
    # 제약 조건 확인 및 위반 시 패널티 부여
    if C < C_min or C > C_max:
        return float('inf')

    
    if np.any(g_i > C_max - sum(L_i)):     
        return float('inf') 

    
    delays = []
    capacities = []
    for i in range(num_phases):
        λ_i = g_i[i] / C
        Q_i = S_i[i] * λ_i
        capacities.append(Q_i)
        X = v[i] / Q_i
        delay = ((C - λ_i) ** 2 / (2 * (1 - λ_i * X))) + (X ** 2 / (2 * v[i] * (1 - X))) - (0.65 * (C / (v[i] ** 2)) ** (1/3) * X ** (2 + 5 * λ_i))
        delays.append(delay)
    
    total_delay = sum(delays)
    total_capacity = sum(capacities)
    
    a1 = 0.3  # 가중치 설정
    a2 = 0.4  # 가중치 설정
    return a1 * total_delay + a2 * (1 - total_capacity)

# 유전 알고리즘 파라미터 설정
population_size = 20
num_generations = 100
mutation_rate = 0.4
crossover_rate = 0.8

# 초기화: 무작위 개체 생성
initial_g_i = np.random.uniform(30, 80, (population_size, num_phases))  # 무작위 초기 위치 설정 (녹색 시간)
initial_L_i = np.random.uniform(1, 5, (population_size, num_phases))        # 무작위 초기 위치 설정 (손실 시간)
population = np.hstack((initial_g_i, initial_L_i)).astype(float)
fitness_scores = np.array([objective_function(ind) for ind in population])

# 선택 연산
def selection(population, fitness_scores):
    probabilities = 1 / (1 + fitness_scores)  # 적합도에 반비례하는 선택 확률 계산
    probabilities /= probabilities.sum()      # 확률 정규화
    selected_indices = np.random.choice(len(population), size=len(population), replace=True, p=probabilities)
    return population[selected_indices]

# 교차 연산
def crossover(parent1, parent2):
    if np.random.rand() < crossover_rate:
        crossover_point = np.random.randint(1, len(parent1))
        child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
        return child1, child2
    else:
        return parent1, parent2

# 돌연변이 연산
def mutate(individual):
    for i in range(len(individual)):
        if np.random.rand() < mutation_rate:
            individual[i] += np.random.uniform(-1, 1)
    return individual

# 유전 알고리즘 실행
for generation in range(num_generations):
    # 선택
    population = selection(population, fitness_scores)
    
    # 교차 및 돌연변이
    new_population = []
    for i in range(0, len(population), 2):
        parent1, parent2 = population[i], population[i + 1]
        child1, child2 = crossover(parent1, parent2)
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))
    population = np.array(new_population)
    
    # 제약 조건을 만족하도록 수정
    for individual in population:
        C = sum(individual[:num_phases]) + sum(individual[num_phases:])
        if C < C_min:
            scaling_factor = C_min / C
            individual[:num_phases] *= scaling_factor
        elif C > C_max:
            scaling_factor = C_max / C
            individual[:num_phases] *= scaling_factor
        
        individual = np.clip(individual, 0, C_max)  # g_i와 L_i가 비현실적으로 크지 않도록 제한
    
    # 적합도 계산
    fitness_scores = np.array([objective_function(ind) for ind in population])

# 최적 신호주기 출력
optimal_params = population[fitness_scores.argmin()]
g_i = optimal_params[:num_phases]
L_i = optimal_params[num_phases:]
optimal_signal_cycle = sum(g_i) + sum(L_i)

print(f"최적 신호주기: {optimal_signal_cycle:.1f}s")
print(g_i)
print(L_i)

end = time.time()
print(end - start)


최적 신호주기: 81.7s
[ 5.         53.67410792  5.        ]
[3.80640985 9.66745526 4.53092382]
0.16256427764892578
